In [ ]:
#%matplotlib inline
#import matplotlib.pyplot as plt
#import graphviz
import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
import kaggle_xgb
import calc_ndcg
#from sklearn.grid_search import ParameterGrid
import pickle
import dataEngr
reload(dataEngr)

Read in just the testing and training data.

In [ ]:
xgbInput = dataEngr.clfInput()
xgbInput.get_sessionsFtr()
xgbInput.users_ftrEng()
xgbInput.one_hot()
#xgbInput.binarize_targets()
xgbInput.split_data(update_trainDf=True)

In [4]:
cv

,test-error-mean,test-error-std,train-error-mean,train-error-std
0,0.088685,0.001540,0.086748,0.000393
1,0.087984,0.001391,0.086076,0.000280
2,0.087624,0.001463,0.085826,0.000297
3,0.087582,0.001476,0.085653,0.000255
4,0.087591,0.001436,0.085539,0.000260
5,0.087420,0.001403,0.085399,0.000254
6,0.087373,0.001349,0.085277,0.000239
7,0.087409,0.001436,0.085160,0.000203
8,0.087391,0.001334,0.085047,0.000234
9,0.087428,0.001331,0.084969,0.000231


In [2]:
%run -i actionsXgb.py

number of columns before one hot encoding 369
number of columns after one hot encoding

Will train until cv error hasn't decreased in 17 rounds.
Stopping. Best iteration: 29


 468


In [ ]:
with open('../xgbmodels/actions2_e13_96n.p') as f:
    bst = pickle.load(f)

In [ ]:
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, xgbInput.allDf.columns[int(feat[1:])], score) for feat, score in features]